In [1]:
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib import RDF, RDFS, OWL, XSD


SCHEMA = Namespace("https://schema.org/")
DCTERMS = Namespace("http://purl.org/dc/terms/")
DBO = Namespace("http://dbpedia.org/ontology/")
SOGIDES = Namespace("http://www.sogides.com/")
SOGIDES_PROP = Namespace("http://www.sogides.com/prop/")
BANQ = Namespace("http://banq.qc.ca/")
WDT = Namespace("http://www.wikidata.org/prop/direct/")
WD = Namespace("http://www.wikidata.org/entity/")

# Chargement des graphes RDF

Nos disposons de cinq graphes RDF distincts:

* Celui extrait des donnnées fournies par Messagerie ADP (séparé en deux: un graphe pour les auteurs et un autre pour les livres)
* Celui extrait à partir du site de l'Infocentre littéraire des écrivains, L’ÎLE (http://recif.litterature.org/)
* Celui construit à partir du Dépôt légal de BAnQ (https://www.donneesquebec.ca/recherche/fr/dataset/publications-recues-en-depot-legal)
* Celui extrait des données fournies par Hurtubise
* Celui extrait des données récupérées par Grégoire sur Babelio(séparé en deux: un graphe pour les auteurs et un autre pour les livres)

In [2]:
grapheAuteursADP = Graph()
grapheAuteursADP.parse('../Graphes/grapheADPAuteurs.rdf')


<Graph identifier=Nb1202f196709467f9e1b459ea9f92b10 (<class 'rdflib.graph.Graph'>)>

In [3]:
grapheLivresADP = Graph()
grapheLivresADP.parse('../Graphes/grapheADPLivres.rdf')


<Graph identifier=N939d9cde8a0c46feab25b1f0b492edd6 (<class 'rdflib.graph.Graph'>)>

In [4]:
grapheILE = Graph()
grapheILE.parse('../Graphes/grapheILE.rdf')

<Graph identifier=Na20525ce9e8646aa98000b97ee606052 (<class 'rdflib.graph.Graph'>)>

In [5]:
grapheDepotLegal = Graph()
grapheDepotLegal.parse('../Graphes/grapheDepotLegal.rdf')

<Graph identifier=N820fa5be4c714f24bd67df07b227b908 (<class 'rdflib.graph.Graph'>)>

In [6]:
grapheHurtubise = Graph()
grapheHurtubise.parse('../Graphes/grapheHurtubise.rdf')

<Graph identifier=Na6bb1f3593864de081adcc2662ef9833 (<class 'rdflib.graph.Graph'>)>

In [7]:
grapheBabelio = Graph()
grapheBabelio.parse('../Graphes/grapheBabelio.rdf')

<Graph identifier=N16af3c42c0024d4db5912c51939d4aa5 (<class 'rdflib.graph.Graph'>)>

In [8]:
def nettoyer_unicode(c):
    liste_codes =  {
        'Ã\xa0': 'à',
        'Ã€': 'À',
        'Ã¢': 'â',
        'Ã‚': 'Â',
        'Ã©': 'é',
        'Ã\x89': 'É',
        'Ã\xa8': 'è',
        'Ã\xaa': 'ê',
        'Ã\x8a': 'Ê',        
        'Ã«': 'ë',
        'Ã®':'î',
        'Ã\x8e':'Î',        
        'Ã¯': 'ï', 
        'Ã´': 'ô',
        'Ã\x94': 'Ô',
        'Ã¹': 'ù', 
        'Ã»': 'û',
        'Å\x93': 'œ', 
        'Â«': '«',
        'Â»': '»',
        'Ã§': 'ç',
        'Ã\x87': 'Ç',
        'Âº': 'º',
        'â\x80\x99': '’',
        'â\x80\xa6' : '…',
    }
    
    for code in liste_codes:
        c = c.replace(code,liste_codes[code])
        
    return c

In [9]:
import re

# def format_isbn(s):
#     """
#     nettoyage du isbn, x-xxxxx-xxx-x (info) --> xxxxxxxxxx
#     :param string: chaine de caractère à normaliser
#     :return: même chaine de caractère normalisée
#     """
#     string = re.sub(r'[^0-9]', '', s)
#     if len(string) == 10:
#         string = '978' + string
#     string = string[:-1] + 'X'
#     return string

# import re

def format_isbn(s):
    """
    nettoyage du isbn, x-xxxxx-xxx-x (info) --> xxxxxxxxxx
    :param string: chaine de caractère à normaliser
    :return: même chaine de caractère normalisée
    """
#     string = s[:-1] + 'X'
    # todo not X probleme FIX 
    string = re.sub(r'\D', '', s)
    if len(string) == 9 or len(string) == 12:
        string += 'X'
    if len(string) == 10:
        string = '978' + string
    return string

print(format_isbn('289006591X'))
print(format_isbn('9872890065913'))
print(format_isbn('987289006591X'))
print(format_isbn('2890065912'))

978289006591X
9872890065913
987289006591X
9782890065912


# Extraction de livres de ADP

In [10]:
qres = grapheLivresADP.query(
    """PREFIX schema: <https://schema.org/>
       SELECT DISTINCT ?livre ?titre ?isbn
       WHERE {
          ?livre a <http://www.sogides.com/classe/Livre> ;
             schema:name ?titre ;
             schema:isbn ?isbn .
       }""")

#adp auteur
#  ?auteur a <http://www.sogides.com/classe/Auteur> ;
#              schema:name  ?nom . 

livres_adp = {}
for r in qres:
    livres_adp[format_isbn(r['isbn'].toPython())] = nettoyer_unicode(r['titre'])

In [11]:
print(len(livres_adp))

15469


# Extraction de livres Dépôt Légal

In [12]:
qres = grapheDepotLegal.query(
     """PREFIX ns1: <https://schema.org/>
        SELECT DISTINCT ?livre ?isbn ?titre
        WHERE {
            ?livre a <http://dbpedia.org/ontology/Book>;
                ns1:name ?titre ;
                ns1:isbn ?isbn .
        }""")

livres_depot = {}
for r in qres:
    livres_depot[format_isbn(r['isbn'].toPython())] = nettoyer_unicode(r['titre'])

In [13]:
print(len(livres_depot))

61570


# Extraction livres Dépôt ILE

In [14]:
qres = grapheILE.query(
     """PREFIX schema: <https://schema.org/>
        SELECT DISTINCT ?livre ?isbn ?titre
        WHERE {
            ?livre a <http://recif.litterature.org/ontologie/classe/oeuvre> ;
                schema:name ?titre ;
                schema:isbn ?isbn .
        }""")


#ile auteur
#            ?auteur a <http://recif.litterature.org/ontologie/classe/ecrivain> ;
#              schema:givenName ?prenom ;
#              schema:familyName ?nom .

livres_ile = {}
for r in qres:
    livres_ile[format_isbn(r['isbn'].toPython())] = nettoyer_unicode(r['titre'])

In [15]:
print(len(livres_ile))

20251


# Extraction livres Hurtubise

In [16]:
qres = grapheHurtubise.query(
     """PREFIX ns1: <http://schema.org/>
        PREFIX hurtubise: <https://distributionhmh.com/onto/>
        SELECT DISTINCT ?livre ?titre ?isbn ?isbnpdf ?isbnepub
        WHERE {
            ?livre a <http://schema.org/Book> ;
                ns1:name ?titre ;
                ns1:isbn ?isbn .
            OPTIONAL {
            ?livre hurtubise:isbnpdf ?isbnpdf } .
            OPTIONAL {
            ?livre hurtubise:isbnepub ?isbnepub }
        }""")

livres_hurtubise = {}
for r in qres:
    livres_hurtubise[format_isbn(r['isbn'].toPython())] = r['titre']
    if r['isbnpdf']:
        livres_hurtubise[format_isbn(r['isbnpdf'].toPython())] = r['titre']
    if r['isbnepub']:
        livres_hurtubise[format_isbn(r['isbnepub'].toPython())] = r['titre']

# Extraction livres Babelio

In [17]:
qres = grapheBabelio.query(
     """PREFIX ns1: <https://schema.org/>
        SELECT DISTINCT ?livre ?titre ?isbn
         WHERE {
            ?livre a <https://schema.org/Book> ;
                ns1:name ?titre ;
                ns1:isbn ?isbn .
        }""")

livres_babelio = {}
print(len(qres))
for r in qres:
    livres_babelio[format_isbn(r['isbn'].toPython())] = r['titre']

1793


# Candidats à l'alignement (gold standard)
Livres confirmés comme étant le même par leur isbn

In [18]:
import re

def normalize(string):
    """
    normalise une chaine de caractère pour faciliter leurs comparaisons
    :param string: chaine de caractère à normaliser
    :return: même chaine de caractère normalisée
    """
    try:
#         string = nettoyer_accents(string)
        string = re.sub(r'^\W+|\W+$', '', string)
        string = re.sub(r'\W', ' ', string)
        string = re.sub(r' +', ' ', string)
        string = re.sub(r',', '', string)
        return string
    except:
        return None

In [19]:
alignement = {}

set_adp_dep = set(livres_adp) & set(livres_depot)
set_adp_ile = set(livres_adp) & set(livres_ile)
set_adp_hur = set(livres_adp) & set(livres_hurtubise)
set_adp_bab = set(livres_adp) & set(livres_babelio)
set_ile_dep = set(livres_ile) & set(livres_depot)
set_ile_hur = set(livres_ile) & set(livres_hurtubise)
set_ile_bab = set(livres_ile) & set(livres_babelio)
set_dep_hur = set(livres_depot) & set(livres_hurtubise)
set_dep_bab = set(livres_depot) & set(livres_babelio)
set_hur_bab = set(livres_hurtubise) & set(livres_babelio)

isbns = set(set_adp_dep ^ set_adp_ile ^ set_adp_hur ^ set_ile_dep ^ set_ile_hur ^ set_ile_bab ^ set_dep_hur ^ set_dep_bab ^ set_hur_bab)
print(len(isbns))
for isbn in isbns:
    
    value = (isbn, '', '', '', '', '')
    value = list(value)
    
    if isbn in livres_adp:
        value[1] = livres_adp[isbn]
    
    if isbn in livres_ile:
        value[2] = livres_ile[isbn]

    if isbn in livres_depot:
        value[3] = livres_depot[isbn]

    if isbn in livres_hurtubise:
        value[4] = livres_hurtubise[isbn]

    if isbn in livres_babelio:
        value[5] = livres_babelio[isbn]

    
    alignement[isbn] = value

13824


In [25]:
print(len(alignement))
print(alignement['9782897230319'])


13824
['9782897230319', '', '', "Mais qu'est-ce que tu fais là, tout seul?", rdflib.term.Literal("Mais qu'est-ce que tu fais là, tout seul?"), '']


In [26]:
import csv

csv_columns = ['ISBN', 'ADP', 'ILE', 'Depot Legal', 'Hurtubise', 'Babelio']

with open('alignement.csv', 'w') as f:
    for key in alignement.keys():
        f.write("%s, %s, %s, %s, %s, %s\n" % (key, normalize(alignement[key][1]),  normalize(alignement[key][2]), normalize(alignement[key][3]), normalize(alignement[key][4]), normalize(alignement[key][5])))


# Création des paires positives

Pour créer des paires positives il nous faut éliminer les 